#### Este baseline se basara en armar un modelo LSTM por cada producto, con una optimizacion de hiper parametros escueta, para poder comparar con futuros experimientos. En caso de que esta alternativa funcione bien, seria recomendable incorporar parametros de optimizacion extra.

#### Imports

In [159]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, RMSprop, SGD
from kerastuner.tuners import BayesianOptimization
from keras import backend as K

In [160]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [161]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


In [162]:
# columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter', 'age', 'y']
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']

#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos desde i-0 hasta i-n, y el valor de "y" sera el valor de "y" 2 meses en el futuro ( i+2 ).

In [163]:
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix >= len(data):
            break
        seq_x, seq_y = data[:i+1, :], data[end_ix, -1]  # y es 2 periodos en el futuro
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X, dtype=object), np.array(y)


Definimos la funcion para crear el modelo LSTM, sobre este se ejecutara la optimizacion bayesiana

In [164]:
def custom_loss(y_true, y_pred):
    penalty = K.mean(K.square(K.minimum(y_pred, 0)), axis=-1)
    mse_loss = K.mean(K.square(y_true - y_pred), axis=-1)
    return mse_loss + penalty

In [165]:
def create_model(activation='relu', units=128, dropout=0.2, learning_rate=0.001, l2_penalty=0.001, depth=3, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units=int(units/2), activation=activation, input_shape=(None, n_features), return_sequences=True, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    for _ in range(depth - 1):
        model.add(LSTM(units=units, activation=activation, return_sequences=True, kernel_regularizer=l2(l2_penalty)))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    
    model.add(LSTM(units=int(units*2), activation=activation, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='relu'))  # Unidades de salida ajustadas a 1 para regresión
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='mse')
    return model

#### Armado de los modelos

In [166]:
n_steps = 2  # número de pasos de tiempo
epochs = 100
batch_size = 32
predictions = []

In [167]:
# Codigo para visualizar como queda la estructura de X e y para 1 producto en particular
# product_ids = final_dataset['product_id'].unique()
# product_id =  product_ids[0]
# product_data = final_dataset[final_dataset['product_id'] == product_id].sort_values(by='periodo')[columns]
# product_data_array = product_data.values
# X, y = prepare_data(product_data_array, n_steps)

# display(product_data)
# display(X)

,plan_precios_cuidados,cust_request_qty,cust_request_tn,close_quarter,y
2628,0,278,101.87181,0,101.87181
2629,0,360,154.27428,0,154.27428
2630,0,360,184.77117,1,184.77117
2631,0,387,237.62667,0,236.05244
2632,0,355,222.28526,0,216.87942
2633,0,395,206.63961,1,206.63219
2634,0,254,137.46271,0,136.60134
2635,0,412,184.16980,0,184.16980
2636,0,350,119.82696,1,119.05470
2637,0,226,146.44026,0,146.44026


array([array([[  0.     , 278.     , 101.87181,   0.     , 101.87181]]),
       array([[  0.     , 278.     , 101.87181,   0.     , 101.87181],
              [  0.     , 360.     , 154.27428,   0.     , 154.27428]]),
       array([[  0.     , 278.     , 101.87181,   0.     , 101.87181],
              [  0.     , 360.     , 154.27428,   0.     , 154.27428],
              [  0.     , 360.     , 184.77117,   1.     , 184.77117]]),
       array([[  0.     , 278.     , 101.87181,   0.     , 101.87181],
              [  0.     , 360.     , 154.27428,   0.     , 154.27428],
              [  0.     , 360.     , 184.77117,   1.     , 184.77117],
              [  0.     , 387.     , 237.62667,   0.     , 236.05244]]),
       array([[  0.     , 278.     , 101.87181,   0.     , 101.87181],
              [  0.     , 360.     , 154.27428,   0.     , 154.27428],
              [  0.     , 360.     , 184.77117,   1.     , 184.77117],
              [  0.     , 387.     , 237.62667,   0.     , 236.05244]

In [168]:
import os


product_ids = final_dataset['product_id'].unique()
    
for product_id in product_ids:
    product_data = final_dataset[final_dataset['product_id'] == product_id].sort_values(by='periodo')[columns]
    
    # Convertir los datos a numpy array
    product_data_array = product_data.values
    
    # Preparar los datos para LSTM
    X, y = prepare_data(product_data_array, n_steps)
    n_features = product_data_array.shape[1]  # Número de características por observación

    model = create_model()

    # Callback para detener el entrenamiento temprano
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

    # Convertir X a una lista de arrays para que funcione con keras tuner
    X_list = [x.tolist() for x in X]    

    max_length = max(len(seq) for seq in X)
    X_padded = np.array([np.pad(seq, ((max_length - len(seq), 0), (0, 0)), 'constant') for seq in X])

    model.fit(X_padded, y, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stopping, reduce_lr], verbose= 0)


    os.makedirs('Models_params', exist_ok=True)
    model.save(f'Models_params/model_product_{product_id}.h5')

    last_record = product_data_array
    last_record = last_record.reshape((1, last_record.shape[0], last_record.shape[1]))
    predicted_y = model.predict(last_record)[0][0]

    # Agregar predicción al resultado
    predictions.append({'product_id': product_id, 'predicted_y': predicted_y})

    print(f'Modelo para el producto {product_id} entrenado y guardado. Predicción a 2 meses: {predicted_y}')

1/1 [==============================] - 3s 3s/step
Modelo para el producto 20001 entrenado y guardado. Predicción a 2 meses: 310.173828125
1/1 [==============================] - 3s 3s/step
Modelo para el producto 20002 entrenado y guardado. Predicción a 2 meses: 58.76602554321289
1/1 [==============================] - 3s 3s/step
Modelo para el producto 20003 entrenado y guardado. Predicción a 2 meses: 15.769316673278809
1/1 [==============================] - 1s 1s/step
Modelo para el producto 20004 entrenado y guardado. Predicción a 2 meses: 0.0
1/1 [==============================] - 3s 3s/step
Modelo para el producto 20005 entrenado y guardado. Predicción a 2 meses: 146.11033630371094
1/1 [==============================] - 3s 3s/step
Modelo para el producto 20006 entrenado y guardado. Predicción a 2 meses: 71.7757797241211
1/1 [==============================] - 3s 3s/step
Modelo para el producto 20007 entrenado y guardado. Predicción a 2 meses: 13.74809455871582
1/1 [==================

In [169]:
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('../../Datasets/predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv.')

Todas las predicciones han sido generadas y guardadas en predictions.csv.
